In [1]:
import torchvision.models as models
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

May try to calculate metrics on every batch instead of calculating at the end of all batches

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [30]:
model =  models.vgg19(weights='IMAGENET1K_V1').to(device)

In [31]:
for param in model.parameters():
    param.requires_grad = False
model.classifier[0].requires_grad = True
model.classifier[3].requires_grad = True

In [32]:
model.classifier[6] = nn.Linear(4096, 15).to(device)

In [33]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [34]:
train_dataset = datasets.ImageFolder("flowers/train", transform=data_transform)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)

In [35]:
val_dataset = datasets.ImageFolder("flowers/val", transform=data_transform)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [36]:
test_dataset = datasets.ImageFolder("flowers/test", transform=data_transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [37]:
criterion = nn.CrossEntropyLoss()

In [38]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [39]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer, num_epoches=5):
    
    val_losses = []
    val_precisions = []
    val_recalls = []
    val_f1_scores = []
    val_accuracies = []
    
    for epoch in range(num_epoches):
        model.train()
        for images, labels in train_dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        val_loss, val_accuracy, val_precision, val_recall, val_f1, _, _ = evaluate(model, val_dataloader, criterion)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        val_precisions.append(val_precision)
        val_recalls.append(val_recall)
        val_f1_scores.append(val_f1)
        
        print(f'Epoch {epoch + 1}/{num_epoches}, Validation Accuracy: {100 * val_accuracy:.2f}%, Validation Loss: {val_loss:.2f}')
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 4, 1)
    plt.plot(val_losses, label='Validation')
    plt.title('Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    metrics = [('Accuracy', val_accuracies),
               ('Precision', val_precisions),
               ('Recall', val_recalls),
               ('F1 Score', val_f1_scores)]

    for i, (metric_name, val_metric) in enumerate(metrics, start=2):
        plt.subplot(1, 4, i)
        plt.plot(val_metric, label='Validation')
        plt.title(f'Validation {metric_name}')
        plt.xlabel('Epochs')
        plt.ylabel(metric_name)
        plt.legend()

    plt.tight_layout()
    plt.show()

In [40]:
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        loss = running_loss / len(dataloader)
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        f1 = f1_score(all_labels, all_preds, average='weighted')
    return loss, accuracy, precision, recall, f1, all_preds, all_labels

In [ ]:
def get_cm(all_preds, all_labels):
    cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(14, 14))
sns.heatmap(cm, annot=True, fmt='d', cbar=False,
            xticklabels=['Aster', 'Calendula', 'California Poppy', "Coreopsis","Daisy","Dandelion",
                         "Iris","Lavender","Lily","Marigold","Orchid","Poppy","Rose","Sunflower","Tulip"], 
            yticklabels=['Aster', 'Calendula', 'California Poppy', "Coreopsis","Daisy","Dandelion",
                         "Iris","Lavender","Lily","Marigold","Orchid","Poppy","Rose","Sunflower","Tulip"], 
            )
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix Heatmap')
plt.show()

In [41]:
train(model, train_dataloader, val_dataloader, criterion, optimizer, 20)

Epoch 1/20, Validation Accuracy: 71.42%, Validation Loss: 0.94
Epoch 2/20, Validation Accuracy: 70.67%, Validation Loss: 0.98
Epoch 3/20, Validation Accuracy: 72.04%, Validation Loss: 0.95
Epoch 4/20, Validation Accuracy: 71.11%, Validation Loss: 0.98
Epoch 5/20, Validation Accuracy: 72.09%, Validation Loss: 1.06
Epoch 6/20, Validation Accuracy: 74.31%, Validation Loss: 0.88
Epoch 7/20, Validation Accuracy: 73.96%, Validation Loss: 0.98
Epoch 8/20, Validation Accuracy: 71.73%, Validation Loss: 1.04
Epoch 9/20, Validation Accuracy: 76.04%, Validation Loss: 0.90


KeyboardInterrupt: 

In [46]:
loss, accuracy, precision, recall, f1, all_preds, all_labels = evaluate(model, test_dataloader, criterion)

KeyboardInterrupt: 

In [ ]:
get_cm(all_preds, all_labels)

In [42]:
model2 = models.vgg19().to(device)

In [43]:
model2.classifier[6] = nn.Linear(4096, 15).to(device)

In [44]:
for param in model2.parameters():
    param.requires_grad = True

In [45]:
optimizer2 = torch.optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)

In [ ]:
train(model2, train_dataloader, val_dataloader, criterion, optimizer2, 10)

In [ ]:
loss, accuracy, precision, recall, f1, all_preds, all_labels = evaluate(model2, test_dataloader, criterion2) 

In [ ]:
get_cm(all_preds, all_labels)